In [1]:
from pathlib import Path
from timeit import repeat

from codetf.models import load_model_pipeline

/home/paul/.conda/envs/codetf/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root_dir = Path.cwd()

In [6]:
def test_runtime(model_name: str, task: str, model_type: str):
    model = load_model_pipeline(model_name=model_name, task=task, model_type=model_type)
    
    def _predict():
        model.predict(['def hello(): print("Hello world")'])

    return repeat(_predict, number=1, repeat=100)

In [7]:
test_runtime("codet5", "sum_python", "base")

[0.0995858409999073,
 0.0976322460001029,
 0.10349974900009329,
 0.10398069500001839,
 0.09988960500004396,
 0.0961179359999278,
 0.09682975000009719,
 0.09947215099987261,
 0.10007587299992338,
 0.1145752340000854,
 0.09794307999982266,
 0.10700170299992351,
 0.1029989830001341,
 0.10891202000016165,
 0.11773066300020218,
 0.12510442600000715,
 0.09836611399987305,
 0.09649235200004114,
 0.09671914099999412,
 0.09962165199999617,
 0.09787681500006329,
 0.09601843400014332,
 0.09843015000001287,
 0.09659469000007448,
 0.09699032899993654,
 0.09604682100007267,
 0.09700200599991149,
 0.09579266499986261,
 0.09876444900010029,
 0.09532335499989131,
 0.11037276899992321,
 0.10538274200007436,
 0.12425645500002247,
 0.10461382400012553,
 0.15426794899985907,
 0.10197256799983734,
 0.0978946420000284,
 0.10240088399996239,
 0.0965277430000242,
 0.10091725300003418,
 0.09803526100017734,
 0.09656351299986454,
 0.09775610099995902,
 0.09796976200004792,
 0.1001700310000615,
 0.095341559999951